This file might not be of practical use, but still interesting to take a look at to facilitate a better understanding of classification behavior for unbalanced data set. Random forest applied to data set (unbalanced).

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#from xgboost.sklearn import XGBClassifier

np.random.seed(0)

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn
import sklearn.grid_search
#from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

import math
from sklearn.metrics import confusion_matrix
from NDCG_score_func import ndcg_score

Loading data X,y and X_test

In [2]:
X= np.load('X.npy')
y= np.load('y.npy')
X_test= np.load('X_test.npy')

### Random Forest

To get a confusion matrix, we need a train data and test data and the true label and predicted value. 

Let's first split the origitnal traiing data into training set and test set.

In [3]:
#Split the origitnal traiing data into training set and test set
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X,y, test_size=0.25, random_state=42)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(160088L, 226L)
(53363L, 226L)
(160088L,)
(53363L,)


Fit random forest classifier with training set and test the performance on test set.

In [4]:
%%time
clf = RandomForestClassifier(max_features=40, n_estimators=120, n_jobs=1, min_samples_split=5)#n_jobs=2 #, min_samples_split=5  max_features=12
clf.fit(X_train,y_train)
y_test_pred = clf.predict(X_test)

Wall time: 3min 57s


In [5]:
#get the labels
Labels = ['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NDF', 'NL', 'PT', 'US', 'other']
confusion= pd.DataFrame(data = confusion_matrix(y_test, y_test_pred),index = Labels, columns=Labels )

__Confusion Matrix__

In the following confusion matrix, the row index represent the true book type, and the columns index represents the predicted booking class.

In [6]:
print confusion

       AU  CA  DE  ES  FR  GB  IT    NDF  NL  PT    US  other
AU      0   0   0   0   0   0   0     64   0   0    77      0
CA      0   0   0   0   1   0   0    169   0   0   215      3
DE      0   0   0   0   0   0   0    146   0   0   134      0
ES      0   0   0   0   0   0   0    261   0   0   264      0
FR      0   0   0   0   1   1   0    633   0   0   607      3
GB      0   0   1   0   0   0   0    299   0   0   312      1
IT      0   0   1   0   1   0   1    359   0   0   324      1
NDF     0   0   2   2  16   7   1  25893   0   0  5085     33
NL      0   0   0   0   1   0   0    117   0   0    94      2
PT      0   0   0   0   0   0   0     22   0   0    29      1
US      1   0   1   1   5   2   3   7620   0   0  7982     20
other   0   0   0   1   2   0   0   1338   0   0  1200      3


In [8]:
total = confusion.sum(axis =1) #the total number of users for each class
NDF = total[7]
non_NDF = total.sum()-NDF
print 'There are %d non-NDF users and %d NDF users in this synthetic test set.'%(non_NDF, NDF)

There are 22324 non-NDF users and 31039 NDF users in this synthetic test set.


Let's see the percentage of recognized users on each category.

In [9]:
recognized = pd.Series(np.diagonal(confusion), index = Labels)
rec_perc = recognized/total *100
rec_perc

AU        0.000000
CA        0.000000
DE        0.000000
ES        0.000000
FR        0.080321
GB        0.000000
IT        0.145560
NDF      83.420858
NL        0.000000
PT        0.000000
US       51.052127
other     0.117925
dtype: float64

__Binarized reading__: In order to make the results more clear, we make a table containing the result of recoginized NDF and recognized non-NDF users. Notice that if a user actually booked Italy is recognized as booking France, it is counted as NOT recognized.

In [10]:
reco_bina = pd.Series(data = [recognized.sum()-recognized['NDF'], recognized['NDF']],index = ['Non-NDF','NDF'])
tot_bina = pd.Series(data = [total.sum()-total['NDF'], total['NDF']],index = ['Non-NDF','NDF'])
rec_perc_bina = reco_bina/tot_bina *100
rec_perc_bina

Non-NDF    35.777638
NDF        83.420858
dtype: float64

In [11]:
print 'The percentage of recognized non-NDF users is only %d'%rec_perc_bina[0]+'%,'+ ' while the percentage of recoginized NDF users is %d'\
                                                          %rec_perc_bina[1]+'%,'

The percentage of recognized non-NDF users is only 35%, while the percentage of recoginized NDF users is 83%,


__Remark__: from the confusion matrix above, it can be seen that due to the dominant amount of 'NDF' and 'US', the predictor tends to predict every thing to be one of them. This is actually due to the fact random forest is trying to maximize purity gain on every step, and therefore the most of samples from dominant classes is classified correctly. __Namely, for a unbalanced data set, classification algorithm is in favor of dominant class.__

We also make a record of accuracy and NDCG score in below.

__Accuracy__ and __NDCG__

In [12]:
sklearn.metrics.accuracy_score(y_test, y_test_pred)

0.6348968386335101

In [13]:
ground_truth = y_test
predictions = clf.predict_proba(X_test)
ndcg_score(ground_truth, predictions, k=5)

0.81853269448565258